# AWS - RDS MySQL
### Paquetes requeridos
Librería de Python necesaria para establecer conexión a una base de datos MySQL

### Credenciales
Para acceder a la BD necesitaremos tres datos:
1. Endpoint/host: la dirección donde se encuentra el servidor
2. Usuario
3. Contraseña

In [34]:
username = "admin"
password = "TheBridgeSchool"
host = "database-2.cvuovpb4vssk.us-east-2.rds.amazonaws.com" 
port = 3306

### Conexion BD

In [35]:
import pymysql
import cryptography

In [36]:
'''
pymysql.cursors.DictCursor para que los resultados que devuelva sean diccionarios,
por defecto devuelve tuplas. Asi podemos acceder por clave a las columnas.
'''

db = pymysql.connect(host = host,
                     user = username,
                     password = password,
                     cursorclass = pymysql.cursors.DictCursor
)

# El objeto cursor es el que ejecutará las queries y devolverá los resultados

cursor = db.cursor()

### Version DB

In [37]:
'''
Este es el engine version de la BD de AWS
fechtone trae la primera linea de la consulta
El execute() devuelve el numero de filas a las que ha afectado la query,
en este caso, devuelve una unica fila.

Execute se guarda en la conexion pero hasta que no hacemos commit
no se ejecutan las queries
de insert de datos y esas cosas...
'''

cursor.execute('SELECT VERSION()')
version = cursor.fetchone()
print(f'MySQL version: {version}')

MySQL version: {'VERSION()': '8.0.28'}


### Creación de DB

In [55]:
# Creamos una BD. Tenemos una instancia de MySQL, pero no una BD
# Los comandos de SQL  se suelen poner en mayuscula
create_db = '''CREATE DATABASE tweets_TheBridge'''
cursor.execute(create_db)



1

In [54]:
# Podemos eliminar la BD
drop_db = '''DROP DATABASE tweets_TheBridge'''
cursor.execute(drop_db)

2

In [56]:
cursor.execute('SHOW DATABASES')
cursor.fetchall()

[{'Database': 'information_schema'},
 {'Database': 'mysql'},
 {'Database': 'performance_schema'},
 {'Database': 'respuestas_GPT'},
 {'Database': 'sys'},
 {'Database': 'tweets_TheBridge'}]

### Creación de tablas

### Seleccionar la BD

In [57]:
# Para usar la BD  recien creada

cursor.connection.commit()
use_db = ''' USE tweets_TheBridge'''
cursor.execute(use_db)

0

In [59]:
# Create two normalized tables, tweets and users
create_table = '''
CREATE TABLE users (
    author_id INTEGER
)
'''
cursor.execute(create_table)

0

In [61]:
# Create two normalized tables, tweets and users

create_table = '''
CREATE TABLE tweets (
  author_id INT,
  created_at TEXT,
  id INT,
  text INT,
  impression_count INT,
  like_count INT,
  quote_count INT,
  reply_count INT,
  retweet_count INT
)

'''
cursor.execute(create_table)

0

In [62]:
# checkear todas las tablas que tiene mi db
cursor.execute('SHOW TABLES')
cursor.fetchall()

[{'Tables_in_tweets_TheBridge': 'tweets'},
 {'Tables_in_tweets_TheBridge': 'users'}]

### Insertar datos

In [64]:
import pandas as pd

In [66]:
username = "admin"
password = "TheBridgeSchool"
host = "database-2.cvuovpb4vssk.us-east-2.rds.amazonaws.com" 
port = 3306

### Guardar los cambios
Hay que ejecutar el commit antes de cerrar la sesión de la BD para que se guarden todos los cambios

In [75]:
db.commit()

### Excepciones
Se recomienda rodear este tipo de sentencias con un try/except, ya que suele ser software productivo y tiene que saber manejar fallos

### Insertar datos de un CSV

In [83]:
df_users = pd.DataFrame(df['author_id'])
df_users.head()

,author_id
0,1151510939105144834
1,570795448
2,1128860148
3,570795448
4,1162694149956603904


In [82]:
df = pd.read_csv('../data/tweets_23&23_@TheBridge.csv', index_col=0)
df.head()

,author_id,created_at,id,text,impression_count,like_count,quote_count,reply_count,retweet_count
0,1151510939105144834,2022-12-29T16:39:19.000Z,1608502937332113408,Empieza el año formándote en una de las profes...,113,4,0,0,1
1,570795448,2022-12-29T12:52:11.000Z,1608445776732246017,Los 4 bootcamps tecnológicos de EDEM y @TheBri...,124,2,0,0,0
2,1128860148,2022-12-29T09:16:11.000Z,1608391421014196224,@EDEMempresarios acoge el 12 de enero un Open ...,49,2,0,0,0
3,570795448,2022-12-27T09:00:57.000Z,1607662809575292929,Si estás buscando una profesión que te cambie ...,218,5,0,0,0
4,1162694149956603904,2022-12-23T12:54:06.000Z,1606271930763354112,En @TheBridge_Tech tenemos un deseo para ti💫\n...,120,1,0,0,1


In [68]:
from sqlalchemy import create_engine

# create sqlalchemy engine

engine = create_engine("mysql+pymysql://{user}:{pw}@{host}/{db}".format(user = username, pw = password, host = host, db = 'tweets_TheBridge'))
# engine = create_engine("mysql+pymysql://my_user:my_password@my_host/my_database")

In [70]:
# insertamos todo el dataframe en users
df_users.to_sql(name='users', con=engine, if_exists= 'append', index=False)


In [72]:
# df into tweets tables
df.to_sql(name='tweets', con=engine, if_exists= 'append', index=False)

In [26]:
db.commit()

In [78]:
# leemos los datos para comprobar que se han ingestado correctamente
sql = '''SELECT * FROM users'''
cursor.execute(sql)
mi_tabla = cursor.fetchall()
mi_tabla

[{'author_id': 2147483647},
 {'author_id': 570795448},
 {'author_id': 1128860148},
 {'author_id': 570795448},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 1175646685},
 {'author_id': 402013017},
 {'author_id': 1128860148},
 {'author_id': 570795448},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 108885393},
 {'author_id': 2147483647},
 {'author_id': 49219057},
 {'author_id': 2147483647},
 {'author_id': 146406912},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 108885393},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 146406912},
 {'author_id': 146406912},
 {'author_id': 146406912},
 {'author_id': 1951117790},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'author_id': 2147483647},
 {'a

### Cerrar la conexión cuando acabemos

In [79]:
# Cerrar conexión mysql
db.close()
# Cerrar conexión sqlalchuemy
engine.dispose()